# Web Scraping - Lab

## Introduction

Now that you've seen a more extensive example of developing a web scraping script, it's time to further practice and formalize that knowledge by writing functions to parse specific pieces of information from the web page and then synthesizing these into a larger loop that will iterate over successive web pages in order to build a complete dataset.

## Objectives

You will be able to:

* Navigate HTML documents using Beautiful Soup's children and sibling relations
* Select specific elements from HTML using Beautiful Soup
* Use regular expressions to extract items with a certain pattern within Beautiful Soup
* Determine the pagination scheme of a website and scrape multiple pages

## Lab Overview

This lab will build upon the previous lesson. In the end, you'll look to write a script that will iterate over all of the pages for the demo site and extract the title, price, star rating and availability of each book listed. Building up to that, you'll formalize the concepts from the lesson by writing functions that will extract a list of each of these features for each web page. You'll then combine these functions into the full script which will look something like this:  

```python
df = pd.DataFrame()
for i in range(2,51):
    url = "http://books.toscrape.com/catalogue/page-{}.html".format(i)
    soup = BeautifulSoup(html_page.content, 'html.parser')
    new_titles = retrieve_titles(soup)
    new_star_ratings = retrieve_ratings(soup)
    new_prices = retrieve_prices(soup)
    new_avails = retrieve_avails(soup)
    ...
 ```

## Retrieving Titles

To start, write a function that extracts the titles of the books on a given page. The input for the function should be the `soup` for the HTML of the page.

In [1]:
def retrieve_titles(soup):
    warning = soup.find('div', class_='alert alert-warning')
    book_container = warning.nextSibling.nextSibling
    titles = [h3.find('a').attrs['title'] for h3 in book_container.findAll('h3')]
    return titles

## Retrieve Ratings

Next, write a similar function to retrieve the star ratings on a given page. Again, the function should take in the `soup` from the given HTML page and return a list of the star ratings for the books. These star ratings should be formatted as integers.

In [12]:
import re

def retrieve_ratings(soup):
    warning = soup.find('div', class_='alert alert-warning')
    book_container = warning.nextSibling.nextSibling
    
    regex = re.compile('star-rating (.*)')
    star_ratings = []
    for p in book_container.findAll('p', {'class':regex}):
        star_ratings.append(p.attrs['class'][-1])
        
    number_map = {'zero':0, 'one':1, 'two':2, 'three':3, 'four':4, 'five':5}
    star_ratings = [number_map[rating.lower()] for rating in star_ratings]
    return star_ratings

## Retrieve Prices

Now write a function to retrieve the prices on a given page. The function should take in the `soup` from the given page and return a list of prices formatted as floats.

In [13]:
def retrieve_prices(soup):
    warning = soup.find('div', class_='alert alert-warning')
    book_container = warning.nextSibling.nextSibling
    
    prices = [p.text for p in book_container.findAll('p', class_='price_color')]
    prices = [float(p[1:]) for p in prices]
    return prices

## Retrieve Availability

Write a function to retrieve whether each book is available or not. The function should take in the `soup` from a given html page and return a list of the availability for each book.

In [4]:
def retrieve_availabilities(soup):
    warning = soup.find('div', class_='alert alert-warning')
    book_container = warning.nextSibling.nextSibling
    
    avails = book_container.findAll('p', class_='instock availability')
    avails = [a.text.strip() for a in avails]
    return avails

## Create a Script to Retrieve All the Books From All 50 Pages

Finally, write a script to retrieve all of the information from all 50 pages of the books.toscrape.com website. 

In [38]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Sandbox below for working out the kinks

# html_page = requests.get('http://books.toscrape.com')
# soup = BeautifulSoup(html_page.content, 'html.parser')

# titles = retrieve_titles(soup)
# ratings = retrieve_ratings(soup)
# prices = retrieve_prices(soup)
# availabilities = retrieve_availabilities(soup)

# df = pd.DataFrame([titles, ratings, prices, availabilities]).transpose()
# df.columns = ['Title', 'Rating', 'Price', 'Availability']

# url = 'http://books.toscrape.com/'
# nextpage = 'cataloge/page-2.html'
# url = url[:url.rfind('/')] + '/' + nextpage
# url
# nextpage = 'page-3.html'
# url = url[:url.rfind('/')] + '/' + nextpage
# url

# html_page = requests.get('http://books.toscrape.com/catalogue/page-49.html')
# soup = BeautifulSoup(html_page.content, 'html.parser')
# print(soup.find('li', class_='next').contents[0]['href'])

In [80]:
df = pd.DataFrame()
url = 'http://books.toscrape.com/'
more_pages = True

while more_pages:
    
    html_page = requests.get(url)
    soup = BeautifulSoup(html_page.content, 'html.parser')
    titles = retrieve_titles(soup)
    ratings = retrieve_ratings(soup)
    prices = retrieve_prices(soup)
    availabilities = retrieve_availabilities(soup)
    this_page = pd.DataFrame([titles, ratings, prices, availabilities]).transpose()
    df = this_page if df.empty else df.append(this_page, ignore_index=True)
    
    if soup.find('li', class_='next'):
        url = url[:url.rfind('/')] + '/' + soup.find('li', class_='next').contents[0]['href']
    else:
        more_pages = False

df.columns = ['Title', 'Rating', 'Price', 'Availability']
df.head()

,Title,Rating,Price,Availability
0,A Light in the Attic,3,51.77,In stock
1,Tipping the Velvet,1,53.74,In stock
2,Soumission,1,50.1,In stock
3,Sharp Objects,4,47.82,In stock
4,Sapiens: A Brief History of Humankind,5,54.23,In stock


In [91]:
df.shape
df.Price = df.Price.astype('float')
df.Rating = df.Rating.astype('int')
df.loc[df['Rating'] >= 4]

,Title,Rating,Price,Availability
3,Sharp Objects,4,47.82,In stock
4,Sapiens: A Brief History of Humankind,5,54.23,In stock
6,The Dirty Little Secrets of Getting Your Dream...,4,33.34,In stock
8,The Boys in the Boat: Nine Americans and Their...,4,22.60,In stock
11,Shakespeare's Sonnets,4,20.66,In stock
...,...,...,...,...
991,Bounty (Colorado Mountain #7),4,37.26,In stock
993,"Bleach, Vol. 1: Strawberry and the Soul Reaper...",5,34.65,In stock
996,"Ajin: Demi-Human, Volume 1 (Ajin: Demi-Human #1)",4,57.06,In stock
997,A Spy's Devotion (The Regency Spies of London #1),5,16.97,In stock


## Level-Up: Write a new version of the script you just wrote. 

If you used URL hacking to generate each successive page URL, instead write a function that retrieves the link from the `"next"` button at the bottom of the page. Conversely, if you already used this approach above, use URL-hacking (arguably the easier of the two methods in this case).

In [ ]:
#Your code here

## Summary

Well done! You just completed your first full web scraping project! You're ready to start harnessing the power of the web!